In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [4]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):                                   
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [5]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q3624078']


In [6]:
description=[]
for i in range(len(listofidqHuman)):
    description.append(research(listofidqHuman[i]))

In [7]:
!pip install transformers==3.3.1
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [8]:
model_path = "./generation model/"

In [9]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [10]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [11]:
import unicodedata

In [12]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format('./word2vec model/GoogleNews-vectors-negative300.bin', binary=True) 

In [13]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport', 'book', 'film']

In [52]:
class Person:
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')

    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteBook | '+self.book
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFilm | '+self.film
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\njob: '+self.job+'\nbook: '+self.book+'\nsport: '+self.sport+'\nmusic: '+self.music+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def respondToFood(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFood()
        print(text)

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def respondToJob(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourJob()
        print(text)

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text

    def respondToMusic(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteMusic()
        print(text)
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text

    def respondToGame(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteGame()
        print(text)
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text

    def respondToSport(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteSport()
        print(text)
        
    def tellYourFavouriteFilm(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite film is '
        elif rand==1:
            text+='I really liked the film '
        elif rand==2:
            text+='I really enjoyed watching the film '
        elif rand==3:
            text+='I like films like '
        text+=self.film
        if rand<4:
            text+='.'
        elif rand==4:
            text+=' was the film that I liked most.'
        return text
    
    def startTopicFilm(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFilm()+' '
        rand=random.randint(0,3)
        if rand==0:
            text+='What is your favourite film?'
        elif rand==1:
            text+='Which film did you like most?'
        elif rand==2:
            text+='Which films do watch?'
        elif rand==3:
            text+='Which film did you enjoy most?'
        print(name+text)
        return text
    
    def respondToFilm(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFilm()
        print(text)
        
    def tellYourFavouriteBook(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite book is '
        elif rand==1:
            text+='I really like the book '
        elif rand==2:
            text+='I enjoyed reading the book '
        elif rand==3:
            text+='I am currently reading '
        elif rand==4:
            text+='I like the book '
        elif rand==5:
            text+='I find '
        text+=self.book
        if rand<3:
            text+='.'
        elif rand==3:
            text+=' and i really like it.'
        elif rand==4:
            text+=' a lot.'
        elif rand==5:
            text+=' is a pretty good book.'
        return text
    
    def startTopicBook(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteBook()+' '
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite book?'
        elif rand==1:
            text+='What do you like to read?'
        elif rand==2:
            text+='Which kind of books do you like?'
        elif rand==3:
            text+='Do you read books?'
        elif rand==4:
            text+='Which book do you like most?'
        print(name+text)
        return text
    
    def respondToBook(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteBook()
        print(text)

    def respond(self, message):
        split=message.split('. ')
        question=split[len(split)-1]
        question=question[:len(question)-1]
        guessedTopic=self.guessTopic(question)
        SWITCH_DICT={
            'sport':self.respondToSport,
            'music':self.respondToMusic,
            'job':self.respondToJob,
            'game':self.respondToGame,
            'food':self.respondToFood,
            'film':self.respondToFilm,
            'book':self.respondToBook
        }
        if not guessedTopic==None:
            SWITCH_DICT.get(guessedTopic)()
        else: 
            #We couldn't find a corresponding topic.
            return -1
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        elif openTopics[rand]=='film':
            doneTopics.append('film')
            return self.startTopicFilm()
        elif openTopics[rand]=='book':
            doneTopics.append('book')
            return self.startTopicBook()
        else:
            return None

In [15]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [16]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [48]:
import time
def startChatInteractive(persons):
    for person in persons: 
        person.introduce()
        time.sleep(2)
    while True:
        message=input('You: ')
        if message=='bye':
            return
        if not message=='':
            random.shuffle(persons)
            for person in persons:
                time.sleep(2)
                res=person.respond(message)
                if res==-1:
                    break

In [18]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [53]:
myPersons=generatePersons(4)

In [55]:
startChat(myPersons, 5)           

Carl:   Hello, I'm Carl Thompson, I'm 36 years old and I come from Dominica.
Ewald:   Good day, I'm Ewald von Kleist, I am 27 years old and I am from South Africa.
Helmuth:   Hi, I am Helmuth von Moltke, I am 76 years old and I am from Trinidad and Tobago.
Albert:   Hello, I'm Albert I, I'm 49 years old and I live in Palau.
Carl:   I really enjoyed watching the film L'avaro. Which film did you enjoy most?
Ewald:   I like films like Oh! Sabella.
Albert:   I really liked the film Zack and Miri Make a Porno.
Helmuth:   The Broken Hearts Club was the film that I liked most.
Albert:   I like listening to children's song. Which type of music do you enjoy most?
Helmuth:   I like listening to Chicago jazz.
Ewald:   My favourite music is G-funk.
Carl:   My favourite music genre is music of Jamaica.
Albert:   My favourite food is Racuchy. What do you like to eat most?
Ewald:   I really enjoy soto betawi.
Carl:   I really enjoy Uja.
Helmuth:   I like to eat Ikadabae.
Helmuth:   My favourite book 

In [56]:
startChatInteractive(myPersons)

Helmuth:   Hey, I am Helmuth von Moltke, I'm 76 years old and I live in Trinidad and Tobago.
Carl:   Hello, I am Carl Thompson, I am 36 years old and I live in Dominica.
Albert:   Hey, my name is Albert I, I am 49 years old and I come from Palau.
Ewald:   Hi, I'm Ewald von Kleist, I'm 27 years old and I live in South Africa.
You: Which sport do you do?
Albert:   I'm a fan of wildwater canoeing.
Carl:   I like to play alindesis.
Ewald:   I like Laser-Run.
Helmuth:   I'm a fan of Japanese-style baseball.
You: To which music do you listen to?
Helmuth:   My favourite music is Chicago jazz.
Albert:   I really enjoy listening to children's song.
Carl:   I usually listen to music of Jamaica.
Ewald:   I like listening to G-funk.
You: What is your favourite dish?
Helmuth:   I really enjoy Ikadabae.
Albert:   My favourite dish is Racuchy.
Carl:   My favourite dish is Uja.
Ewald:   I love soto betawi.
You: bye


In [27]:
generateDescriptions(myPersons)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


The country of Barbados is the country that Joe Johnson, who has a career of working at the height of 53.
Joseph's occupation is honorary consul.
The US Navy's favourite book is the US Navy Navy of Operated in 1941-43.
bobsleigh is a favourite sport of Joseph.
samba is a favourite of Joseph.
The favourite of Joseph is the Forces of Nature.
Flesh is a favourite of Joseph, which is also where the dish flesh is from.
Fried rice with salted fish and chicken are the main ingredients of Joseph.


John the Good is a 61 year old national from the country of Haiti.
John is a baseballer.
The book The Etymologicon is John's favourite book.
The favourite sport of John is the mountain bike.
The canzona is a favourite of John.
The favourite book of John is inside Man.
The favourite games of John are the Patchwork.
The favourite food of John is bubur pedas.


Ivan Ilic is a 27 year old dish from Australia.
The ragpicker, Ivan, is a type of employee.
The Newman's bird of the country, Ivan, is a favour

In [28]:
for person in myPersons:
    person.showProperties()

name: Joseph Johnson
age: 53
country: Barbados
job: honorary consul
book: US Marine Corps Pacific Theater of Operations 1941-43
sport: bobsleigh
music: samba
film: Forces of Nature
game: Flesh & Blood
food: Fried Rice with Salted Fish and Chicken

name: John the Good
age: 61
country: Haiti
job: baseball commentator
book: The Etymologicon
sport: mountain bike trials
music: canzona
film: Inside Man
game: Patchwork
food: bubur pedas

name: Ivan Ilic
age: 27
country: Australia
job: ragpicker
book: Newman's birds of southern Africa
sport: vert skateboarding
music: trivial music
film: Trouble with the Curve
game: Tapa
food: cireng

name: Henry III
age: 57
country: Moldova
job: official receiver
book: Modern Rationalism
sport: deaf floorball
music: easy listening
film: Sleepless in Seattle
game: mecanicartes
food: Nasi Kapau

